# [실습] Langchain으로 시장조사 문서 기반 챗봇 만들기 - PDF

## 실습 목표
---
[실습] LangChain을 활용해서 입력된 문서를 요약해서 Context로 활용하는 챗봇을 개발합니다.

## 실습 목차
---

1. **시장조사 문서 벡터화:** RAG 챗봇에서 활용하기 위해 시장조사 파일을 읽어서 벡터화하는 과정을 실습합니다.

2. **RAG 체인 구성:** 이전 실습에서 구성한 미니 RAG 체인을 응용해서 간단한 시장 조사 문서 기반 RAG 체인을 구성합니다.

3. **챗봇 구현 및 사용:** 구성한 RAG 체인을 활용해서 시장조사 문서 기반 챗봇을 구현하고 사용해봅니다.

## 실습 개요
---
RAG 체인을 활용해서 시장조사 문서 기반 챗봇을 구현하고 사용해봅니다.

## 0. 환경 설정
- 필요한 라이브러리를 불러옵니다.

In [1]:
from langchain.document_loaders import PyPDFLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

- Ollama를 통해 Mistral 7B 모델을 불러옵니다.

In [2]:
!ollama pull mistral:7b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         pulling manifest 
pulling ff82381e2bea... 100

## 1. 시장조사 문서 벡터화
- RAG 챗봇에서 활용하기 위해 시장조사 파일을 읽어서 벡터화하는 과정을 실습합니다.

먼저, mistral:7b 모델을 사용하는 ChatOllama 객체와 OllamaEmbeddings 객체를 생성합니다.

In [3]:
llm = ChatOllama(model="mistral:7b")
embeddings = OllamaEmbeddings(model="mistral:7b")

다음으로, 시장조사 PDF 문서를 불러와서 벡터화 해보겠습니다.
- 한국소비자원의 2022년 키오스크(무인정보단말기) 이용 실태조사 보고서를 활용했습니다
  - https://www.kca.go.kr/smartconsumer/sub.do?menukey=7301&mode=view&no=1003409523&page=2&cate=00000057
- 이 실태조사 보고서는 2022년 키오스크의 사용자 경험, 접근성, 후속 조치에 대해 논의하는 보고서입니다. 
- 이를 활용해서 키오스크를 어떻게 세일즈 할 수 있을지 아이디어를 제공하는 챗봇을 만들어야 하는 상황이라고 가정해 봅시다.

먼저, LangChain의 `PyPDFLoader`를 활용해서 시장조사 보고서의 텍스트를 추출하고, 페이지 별로 `Document`를 생성하여 저장합니다.

In [4]:
doc_path = "docs/키오스크(무인정보단말기) 이용실태 조사.pdf"
loader = PyPDFLoader(doc_path)
docs = loader.load()

생성된 Document의 수를 확인해봅시다.

In [5]:
print(len(docs))

59


다음으로, 각 Document의 길이를 확인해봅시다.

In [6]:
doc_len = [len(doc.page_content) for doc in docs]
print(doc_len)

[86, 6679, 6011, 5915, 1201, 908, 676, 1289, 821, 1154, 1439, 447, 1031, 1178, 514, 1083, 968, 1119, 1006, 1094, 978, 916, 1230, 862, 680, 1251, 1433, 1290, 729, 1170, 1011, 598, 733, 966, 934, 1195, 514, 1210, 777, 635, 651, 771, 837, 397, 953, 877, 548, 1022, 1198, 1183, 1230, 838, 533, 1255, 1231, 1894, 777, 798, 662]


1천자 미만의 문서도 있지만, 6천자가 넘는 문서도 있는 것을 확인할 수 있습니다. 이대로 그냥 사용할 경우, Context가 너무 길어져 오히려 성능이 낮아질 수도 있습니다.

우선은 이대로 RAG 체인을 구성해 봅시다.

## 2. RAG 체인 구성
RAG 체인을 구성하기 위해 `Document`를 `OllamaEmbeddings`를 활용해 벡터로 변환하고, FAISS DB를 활용하여 저장합니다.
- 변환 및 저장 과정은 약 3분 정도 소요됩니다.

In [7]:
vectorstore = FAISS.from_documents(
    docs,
    embedding=embeddings
)

In [8]:
db_retriever = vectorstore.as_retriever()

이전 실습에서 구성한 미니 RAG Chain과 비슷하게 Chain을 구성해 봅시다.
- 지난 실습과 달리 이번 챗봇의 역할은 마케터를 위한 챗봇으로 고정했으므로, 역할을 별도로 인자로 전달할 필요가 없습니다.
- `RunnablePassthrough()`는 Chain의 이전 구성 요소에서 전달된 값을 그대로 전달하는 역할을 수행합니다.

In [9]:
def get_retrieved_text(docs):
    result = "\n".join([doc.page_content for doc in docs])
    return result

def init_chain():
    messages_with_contexts = [
        ("system", "당신은 마케터를 위한 친절한 지원 챗봇입니다. 사용자가 입력하는 정보를 바탕으로 질문에 답하세요."),
        ("human", "정보: {context}.\n{question}."),
    ]

    prompt_with_context = ChatPromptTemplate.from_messages(messages_with_contexts)

    # 체인 구성
    # context에는 질문과 가장 비슷한 문서를 반환하는 db_retriever에 get_retrieved_text를 적용한 chain의 결과값이 전달됩니다.
    qa_chain = (
        {"context": db_retriever | get_retrieved_text, "question": RunnablePassthrough()}
        | prompt_with_context
        | llm
        | StrOutputParser()
    )
    
    return qa_chain

In [10]:
qa_chain = init_chain()

Chain 구성이 완료되었습니다.

## 3. 챗봇 구현 및 사용
- 구성한 RAG 체인을 활용해서 시장조사 문서 기반 챗봇을 구현하고 사용해봅니다.

방금 구현한 RAG Chain을 사용해서 시장조사 문서 기반 챗봇을 구현해볼 것입니다. 

그 전에, 별도로 RAG 기능을 추가하지 않은 LLM과 답변의 퀄리티를 비교해 봅시다.

In [11]:
messages_with_variables = [
    ("system", "당신은 마케터를 위한 친절한 지원 챗봇입니다."),
    ("human", "{question}."),
]
prompt = ChatPromptTemplate.from_messages(messages_with_variables)
parser = StrOutputParser()
chain = prompt | llm | parser

In [12]:
print(chain.invoke("키오스크 관련 설문조사 결과를 알려줘"))

 안녕하세요! 키오스크 관련 설문조사의 일부 결과를 소개해드리겠습니다.

1. 키오스크가 고객 경험을 향상시켰는지에 대한 답변:
    - 매우 긍정적: 35%
    - 긍정적: 48%
    - 중립: 9%
    - 부정적: 2%
    - 매우 부정적: 6%

2. 키오스크가 고객의 문제를 해결하는데 도움이 되었는지에 대한 답변:
    - 매우 긍정적: 40%
    - 긍정적: 45%
    - 중립: 6%
    - 부정적: 2%
    - 매우 부정적: 7%

3. 키오스크를 사용하는 이유:
    - 시간 절약: 45%
    - 편하고 빠르다: 38%
    - 신체적인 부담 없음: 10%
    - 기타 (예: 안전, 개인정보 보호, 일찍 또는 늦게 이용 가능): 7%

4. 키오스크를 사용하면서 발생한 문제점:
    - 시스템에러/장애: 10%
    - 기술적인 문제: 8%
    - 지시문/설명이 부족함: 7%
    - 키오스크의 위치가 불편함: 6%
    - 기타 (예: 장애물, 소음, 망각): 69%

5. 키오스크를 더욱 향상시킬 수 있는 방안에 대한 의견:
    - 명확한 지시문 제공: 28%
    - 기술적인 개선: 24%
    - 키오스크 설치 위치 개선: 17%
    - 간편하고 빠르게 문제 해결이 가능한 지원 제공: 16%
    - 기타 (예: 소음 차단, 추가 기능): 15%


In [13]:
print(qa_chain.invoke("키오스크 관련 설문조사 결과를 알려줘"))

 주요 키오스크(무인정보단말기) 관련 설문 조사 결과는 다음과 같습니다:

1. 키오스크 이용 상황에 대한 응답자의 인식과 생각:
   - 전반적으로, '외식업'에서 가장 많이 이용하는 키오스크를 사용하였다고 보고되었습니다.
   - 피해 경험의 유형으로, 강제 이용, 취소 불가, 변경 불가, 주문 실수, 상품 미제공가 있었는데 '외식업'에서는 각 항목별로 높은 비율을 보였습니다.

2. 키오스크 관련 피해 경험 여부:
   - [그림5-2-1]에서는 업종별로 키오스크 관련 피해 경험 여부와 유형을 나타내고 있습니다. '외식업'에서 피해 경험 가능성이 높았으며, 강제 이용, 취소 불가, 변경 불가, 주문 실수, 상품 미제공가 있었습니다.

3. 피해 유형별로의 특정 업종 분석:
   - [표5-2-6]에서는 업종별로 피해 경험 유형을 나타내고 있으며, '외식업'이 높은 비율을 보입니다. 강제 이용(92.5%), 취소 불가(87.0%), 변경 불가(77.2%), 주문 실수(71.4%), 상품 미제공(73.3%)이 있었습니다.

4. 피해 유형별로의 피해 발생 원인:
   - 피해 유형별로 피해 발생 원인을 조사하면, 강제 이용은 매장에 직원이 있는데도 키오스크로만 거래가 가능하다고 안내함으로 발생합니다.
   - 취소 불가는 주문 후 취소가 불가능함으로 발생했습니다.
   - 변경 불가는 결제 전 장바구니에 담긴 상품 또는 서비스를 변경하지 못함으로 발생합니다.
   - 주문 실수는 주문 실수를 인지하지 못해 주문한 것과 다른 상품‧서비스를 받음으로 발생했습니다.
   - 상품 미제공은 기기 오류 등으로 상품‧서비스를 받지 못했는데도 결제가 됨으로 발생합니다.


일반 체인은 아무런 출처가 없는 답변을 생성한 반면, RAG 기능을 추가한 챗봇은 데이터를 기반으로 상대적으로 정확한 답변을 하는 것을 확인할 수 있습니다. 

이제 챗봇을 한번 사용해 봅시다.

In [ ]:
qa_chain = init_chain()
while True:
    question = input("질문을 입력해주세요 (종료를 원하시면 '종료'를 입력해주세요.): ")
    if question == "종료":
        break
    else:
        result = qa_chain.invoke(question)
        print(result)

저희는 이전 챕터에서 구현한 챗봇이 가지고 있는 문제점 중 '문서나 데이터 기반 추론이 불가능하다.'를 완화했습니다.

또한, 지금 구성한 챗봇은 UI가 없고 단순 표준 입출력 만을 사용합니다. 5챕터에서 Streamlit을 활용해 ChatGPT와 비슷한 웹 챗봇 어플리케이션을 제작해 볼 것입니다.